# Manuscript Utils
This notebook contains useful functions for generating figures in the paper.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
from pylab import *
import os
from typing import Tuple, List

from dataclasses import dataclass

import sys
sys.path.append("../../python_src/")
from figures_utils import generate_experimental_df

# Global Variables and Replacement Dictionaries

In [ ]:
name_replace_dict = {
    "biobakery3": "bioBakery3",
    "biobakery4": "bioBakery4",
    "bio4": "bioBakery4",
    "jams": "JAMS",
    "wgsa2": "WGSA2",
    "wgsa": "WGSA2",
    "woltka": "Woltka",
    "wol": "Woltka",
}

left_join_replace_dict = {"species_exp": "species", "genus_exp": "genus", "RA_obs": "RA"}

@dataclass
class LegendAddition:
    label: str
    color: str
    marker: str
    alpha: float

FP = LegendAddition("False Positive", "grey", "|", 0.5)
UNCLASS = LegendAddition("Unclassified", "#110a70", "x", 0.5)
ASSIGN_HIGHER = LegendAddition("Assigned Higher", "#f22e2e", "O", 0.5)

@dataclass
class Columns:
    false_positive: bool
    unclass: bool
    assigned_higher: bool

# If this taxonomy number comes into use, then it will be in the far, far future
ASSIGNED_HIGHER_ID = 999999999
UNCLASS_ID = 12908

## Expected Plotting
Stacked bar plots with tabulated relative abundances.

In [ ]:
def get_all_expected(root_dir: str, rank="Genus") -> pd.DataFrame:
    combined_expected = pd.DataFrame()
    for root, dirs, files in os.walk(root_dir):
        for f in files:
            # print("files: ", files)
            if f"expected_{rank.lower()}_annotated" in f and f.endswith(".csv"):
                # print(root, f)
                df = pd.read_csv(os.path.join(root, f), index_col=0, names=[rank, 'RA', "TAX_ID"], header=0)
                df["Source"] = root.split("/")[-1]

                # Files are of s#_expected.csv, so we can split on the underscore and take the first part.
                df["SampleID"] = f.split("_")[0]

                # Pipeline for easier grouping later.
                df['Pipeline'] = "Expected"

                combined_expected = pd.concat([combined_expected, df], axis=0)

    return combined_expected

def get_all_relabund_files(root_dir: str, rank="genus") -> pd.DataFrame:
    """
    Parameters:
        root_dir: str 
            The root directory to search for relabund files.
        rank: str
            The taxonomic rank to search for. Default is "genus".
    Returns:
        relabund_files: pd.DataFrame

    Traverses the BASE ROOT directory and searches for ALL {rank}_relabund_annotated files. These are concatenated. \\
    Then, it then returns a dataframe with the sample ID and the path to the relabund file.
    """
    combined_df = pd.DataFrame()
    for root, dirs, files in os.walk(root_dir):
        for f in files:
            if f"{rank.lower()}_relabund_annotated" in f and f.endswith(".csv"):
                # print(root, f)
                p = os.path.join(root, f)
                exp = generate_experimental_df(p, rank)
                exp["Pipeline"] = os.path.dirname(p).split('/')[-1]

                # Add a column to the experimental dataframe with the pipeline name.
                exp['Source'] = os.path.dirname(p).split('/')[-2]

                # Add sampleID to the experimental dataframe.
                exp['SampleID'] = os.path.basename(p).split('_')[0]
                # display(exp.head(10))

                # Add the experimental dataframe to the combined dataframe.
                combined_df = pd.concat([combined_df, exp], axis=0)

    # Ensure that the RA column is a float.
    combined_df['RA'] = combined_df['RA'].astype(float)

    return combined_df

# expected = get_all_expected("../../pipelines/", rank="Genus")
# display(expected.head(50))
# # Print the number of occurences in Source
# expected["Source"].value_counts()

In [ ]:
def plot_pies(expected: pd.DataFrame):
    for source, source_df in expected.groupby("Source"):
        for sample, sample_df in source_df.groupby("SampleID"):
            plot_df = sample_df.loc[sample_df["RA"] > 0]

            plot_df[["RA"]].T.plot(kind="barh", stacked=True, figsize=(20, 10))
            plt.show()

            plt.pie(plot_df["RA"], labels=plot_df.index, autopct='%1.1f%%')
            plt.show()

            # sns.barplot(x='SampleID', y='RA', hue="Genus", data=plot_df)
            # plt.show()

## Creation of comparative stacked bar plots or heatmaps.

We want to create a figure that summarizes the relative abundances of the expected with comparison to each pipeline.

In [ ]:
def get_legend_and_add_entry(ax: plt.Axes, c_objs: list[LegendAddition], contract: int = 15) -> Tuple[plt.Axes, str]:
    """
    Parameters:
        ax: plt.Axes
            The axes to add the legend entry to.
        c_objs: list
            A list of LegendAddition objects.
        contract: int
            The number of objects to contract the label to. Default is 15.
    Returns:
        ax: plt.Axes
            The axes with the legend entry added.
    Gets the legend from the axes and adds a new entry to it. \\
    """
    # Get the legend from the axes.
    handles, labels = ax.get_legend_handles_labels()

    if len (handles) >= contract:
        # We want only the top 15 entries.
        handles = handles[:contract]

    # Add the new legend entry.
    for obj in c_objs:
        handles.append(mpatches.Patch(color=obj.color, label=obj.label, hatch=obj.marker, alpha=obj.alpha))

    # Set the legend.
    ax.legend(handles=handles, loc="upper left", bbox_to_anchor=(1, 1), fontsize=16)

    return ax

In [ ]:
def add_row(df: pd.DataFrame, rank: str, name: str, value: any, taxid: float) -> pd.DataFrame:
    """
    Adds a row with the given value and name to the dataframe.
    """
    df.reset_index(inplace=True)
    df.index.names = [rank]

    new_row = pd.DataFrame({rank: name, "RA": value, "TAX_ID": taxid}, index=[0])
    df = pd.concat([df, new_row], axis=0)

    df = df.set_index(rank)

    return df

def fill_df(df: pd.DataFrame, src: str, rank: str, row_name: str = "Unclassified") -> Tuple[pd.DataFrame, bool]:
    """
    Parameters:
        df: pd.DataFrame
            The dataframe to fill.
        src: str
            The source of the dataframe.
        rank: str
            The taxonomic rank of the dataframe.
        row_name: str
            The name of the row which will fill. Default is "Unclassified".
    Returns:
        df: pd.DataFrame
            The dataframe with the row filled.
        filled: bool
            Whether or not the row was filled.
    
    Fills the input dataframe in the RA column up to 100% with a row of the input row_name. \\
    """
    # We need to get the sum of the RA column for the dataframe.
    # If it it less than 1, we need to fill the difference with a row called "Unclassified".
    df.reset_index(inplace=True)

    # Get the sum of the RA column.
    sum_ra = df["RA"].sum()
    diff = 1 - sum_ra
    # print(diff)

    # If the difference is less than zero, we do not need to add anything.
    if diff <= 0:
        df.set_index(rank, inplace=True)
        return df, False

    # Rename index to rank.
    df.index.names = [rank]

    # If the difference is greater than 0, we need to add a row.
    # new_row = pd.DataFrame({rank: "Unclassified", "RA": diff, "TAX_ID": "Unclassified", "Source": src, "SampleID": sampleID, "Pipeline": "wgsa2"}, index=[0])
    new_row = pd.DataFrame({rank: row_name, "RA": diff}, index=[0])
    df = pd.concat([df, new_row], axis=0)

    # print(df["RA"].sum())
    # display(df)

    df.set_index(rank, inplace=True)

    return df, True

In [ ]:
def make_inverse_range(n: int) -> List[int]:
    # We want to take n (ex. 1) and make a range of [-2].
    # For n = 2, we want [-3, -2].
    # For n = 3, we want [-4, -3, -2].
    lst = []
    for i in range(n):
        lst.append(-i - 2)

    return reversed(lst)

def fix_bar_colors(ax: plt.Axes, labels_list: List[Tuple[str, str, str]]):
    """
    Changes the last bar in the ax to be black. We also need to change the second to last color to a hatched red? \\
    This is done because the legend entry "False Positive" is black and the last bar needs to match. Same for the "Unclassifed".

    Parameters:
        ax: plt.Axes
            The axes to change the color of.
        labels_list: List[Tuple[str, str, str]]
            A list of tuples containing the labels of the bars. The first index is the label name, the second is whether is it present or not, and the third is the color class.
            The order is 
    """
    children = ax.get_children()

    filtered = filter(lambda x: isinstance(x, matplotlib.patches.Rectangle), children)
    filtered_list = list(filtered)

    def fix_color(rect, props: LegendAddition):
        rect.set_facecolor(props.color)
        rect.set_hatch(props.marker)
        rect.set_alpha(props.alpha)

    # false_p = filtered_list[-2]
    # unclass = filtered_list[-3]
    labels_list = [x for x in labels_list if x[1]]
    n = len(labels_list)

    idxs = make_inverse_range(n)

    for c, i in enumerate(make_inverse_range(n)):
        fix_color(filtered_list[i], labels_list[c][2])

    # filtered2 = filter(lambda x: isinstance(x, matplotlib.patches.Rectangle), children)
    # list(filtered2)[-3].set_facecolor("#110a70").set_alpha(0.5)


In [ ]:
def reorganize_columns(df: pd.DataFrame, cols_list: List[Tuple[str, bool]]):
    """
    Reorganizes the columns into the proper order so that colors may be assigned to the correct bars.
    Parameters:
        df: The dataframe to reorganize.
        cols_list: A list of tuples. The first element is the column name and the second is a boolean indicating if the column is present.
    """
    cols = list(df.columns.values)    

    reorg_list = [x[0] for x in cols_list if x[1]]

    for label in reorg_list:
        cols.pop(cols.index(label))
    
    return df[cols + reorg_list]

In [ ]:
def add_plot_decorations(ax, n_rows):
    """
    This function adds a box around the pipelines and adds a textbox with the word "Observed" at 90 degrees.
    """
    # We now need to add an external rectangle to the first plot, the one enclosing the rest.
    autoAxis = ax[0].axis()
    rec = Rectangle((autoAxis[0]-0.15,autoAxis[2]),(autoAxis[1]-autoAxis[0]+0.151),(autoAxis[3]-autoAxis[2]),fill=False,lw=3)
    rec = ax[0].add_patch(rec)
    rec.set_clip_on(False)

    # Now, from the second axis down.
    autoAxis = ax[1].axis()
    box_height = autoAxis[3] - autoAxis[2] + 0.205
    height_mult = n_rows - 2
    # Distance between plots.
    rec = Rectangle((autoAxis[0]-0.15,autoAxis[2]-(box_height*height_mult)),(autoAxis[1]-autoAxis[0]+0.151),(autoAxis[3]-autoAxis[2]+(box_height*height_mult)),fill=False,lw=3)
    rec = ax[1].add_patch(rec)
    rec.set_clip_on(False)

    pts = rec.get_bbox()

    ax[1].text(pts.x0, pts.y0 / 2, "Observed", fontsize=24, ha="right", va="bottom", rotation=90)

    plt.show()

In [ ]:
def plot_expected(ax: plt.Axes, df: pd.DataFrame, max_legend_entries: int = 15):
    """
    ## Parameters:
        ax: The axis to plot on.
        df: The dataframe to plot (should be the expected dataframe).
    
    Plots the expected values for the given ax.
    """
    group_T = df[["RA"]].T
    group_T.index = ["Expected"]
    group_T.plot.barh(stacked=True, ax=ax[0], alpha=0.5, legend=True, fontsize=20)

    ax[0].set_xlim(0, 1.01)

    ax[0] = get_legend_and_add_entry(ax[0], [UNCLASS, FP, ASSIGN_HIGHER], max_legend_entries)

In [ ]:
def add_rows_to_expected(df: pd.DataFrame, rank: str) -> pd.DataFrame:
    df = add_row(df, rank, "Assigned Higher", 0.0, ASSIGNED_HIGHER_ID)
    df = add_row(df, rank, "Unclassified", 0.0, UNCLASS_ID)

    return df

In [ ]:
def plot_one_to_one(smpl_df: pd.DataFrame, expected: pd.DataFrame, src: str, smpl: str, rank: str): 
    n_rows = len(smpl_df["Pipeline"].unique()) + 1

    fig, ax = plt.subplots(nrows=n_rows, ncols=1, figsize=(20, 10), sharex=True)
    fig.suptitle(f"Relative Abundances between Observed and Expected for {src} {smpl} at {rank.capitalize()} Level", fontsize=16, y=0.92)
    
    ax_c = 1

    grouped_expected = expected.loc[(expected["Source"] == src) & (expected["SampleID"] == smpl) & (expected["RA"] > 0)]
    sorted_expected = grouped_expected.copy().sort_index()
    # display(sorted_expected)
    plot_expected(ax, sorted_expected, 30)    

    # display(sorted_expected)
    sorted_expected = add_rows_to_expected(sorted_expected, rank)
    display(sorted_expected)

    # Get the TAX_ID values from the expected as a set.
    # We add 12908 becasue that is the TAX_ID for "Unclassified". We also add the assigned_higher ID (9 9s).
    # expected_tax_ids_set = set(sorted_expected["TAX_ID"].values)
    # expected_tax_ids_set.add(12908)
    # expected_tax_ids_set.add(ASSIGNED_HIGHER_ID)

    for pl, pl_df in smpl_df.groupby("Pipeline"):
        # Get observed total.
        observed_total = pl_df["RA"].sum()

        # The difference between 1 and the observed total is assigned higher.
        assigned_higher_ra = 0
        assigned_higher_bool = False

        if pl != "woltka":
            assigned_higher_ra = 1 - observed_total
            assigned_higher_bool = assigned_higher_ra > 1e-6

        # Add the assigned higher to the dataframe if it is greater than 1 ppm.
        if assigned_higher_bool:
            pl_df = add_row(pl_df, rank, "Assigned Higher", assigned_higher_ra, ASSIGNED_HIGHER_ID)

        # Select the rows from the observed that are in the expected set.
        # left_joined = pd.merge(grouped_expected, pl_df, how="left", on=["TAX_ID"], suffixes=("_obs", "_exp"))
        left_joined = sorted_expected.reset_index().merge(pl_df.reset_index(), how="left", on=["TAX_ID"], suffixes=("_exp", "_obs"))
        # pl_df = pl_df.loc[pl_df["TAX_ID"].isin(expected_tax_ids_set)]

        left_joined = left_joined.loc[(left_joined["RA_exp"].notna()) | (left_joined[f"{rank}_exp"] == "Unclassified") | (left_joined[f"{rank}_exp"] == "Assigned Higher")]
        left_joined = left_joined[[f"{rank}_exp", "RA_obs", "TAX_ID"]].fillna(0)
        left_joined.rename(left_join_replace_dict, axis=1, inplace=True)
        left_joined.set_index(rank, inplace=True)

        # display(pl_df)

        # Find the index value of the row with TAX_ID 12908.
        unclassified_present = 12908 in left_joined["TAX_ID"].values

        unclassified_label = None
        if unclassified_present:
            unclassified_label = left_joined.loc[left_joined["TAX_ID"] == 12908].index[0]

        left_joined, fp_filled = fill_df(left_joined, src, rank, "False Positive")
        # display(left_joined.head(10))
        T = left_joined[["RA"]].T

        # Sort the columns in alphabetical order.
        # T = T.reindex(sorted(T.columns), axis=1)

        # We need to remove the column "Unclassified" from the expected dataframe and remove the 'False Positive' column from the observed dataframe.
        # Then append these to the end of the df in the order "Unclass" then "False Positive", then Assign Higher.
        reorg_list = [(unclassified_label, unclassified_present, UNCLASS), ("False Positive", fp_filled, FP), ("Assigned Higher", assigned_higher_bool, ASSIGN_HIGHER)]

        T = reorganize_columns(T, reorg_list)
        
        # Change the index to the pipeline name.
        T.index = [name_replace_dict[pl]]

        display(T)

        ax[ax_c].set_xlim(0, 1.01)
        T.plot.barh(alpha=0.5, ax=ax[ax_c], legend=False, stacked=True, fontsize=20)
        # ax[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))

        fix_bar_colors(ax[ax_c], reorg_list)

        ax[ax_c].set_xlabel("Relative Abundance", fontsize=24, labelpad=20)

        ax_c += 1

    add_plot_decorations(ax, n_rows) 

    pdf_output.savefig(fig, bbox_inches="tight", dpi=300)

In [ ]:
# Now, we want the charts to represent the left join of the expected and observed dataframes.
# This is to reduce the number of taxa.
pdf_output = PdfPages("pipeline_plotting.pdf")

def stacked_left_join(rank: str):
    expected = get_all_expected("../../pipelines/", rank=rank)
    observed = get_all_relabund_files("../../pipelines/", rank=rank)

    for src, src_df in observed.groupby("Source"):
        print(src)
        if src == "gut" or src == "tongue":
            continue

        if src == "bmock12" or src == "camisimGI" or src == "nist":
            groups = src_df.groupby("SampleID")
            for smpl, smpl_df in groups:
                print("Sample: ", smpl)
                plot_one_to_one(smpl_df, expected, src, smpl, rank)
                
        else:
            groups = src_df.groupby("Pipeline")

            fig, ax = plt.subplots(nrows=len(groups)+1, ncols=1, figsize=(20, 10), sharex=True)
            fig.suptitle(f"Average Relative Abundances between Observed and Expected for {src} at {rank} Level", fontsize=20, y=0.95)
            ax_c = 1

            grouped_expected = expected.loc[(expected["Source"] == src) & (expected["RA"] > 0)]
            grouped_expected = grouped_expected.copy().sort_index()
            plot_expected(ax, grouped_expected, 30)

            sorted_expected = add_row(grouped_expected, rank, "Unclassified", 0.0, UNCLASS_ID)

            for pl, pl_df in groups:
                left_joined = sorted_expected.reset_index().merge(pl_df.reset_index(), how="left", on=["TAX_ID"], suffixes=("_exp", "_obs"))

                left_joined = left_joined.loc[(left_joined["RA_exp"].notna()) | (left_joined[f"{rank}_exp"] == "Unclassified")]
                left_joined = left_joined[[f"{rank}_exp", "RA_obs", "TAX_ID"]].fillna(0)
                left_joined.rename(left_join_replace_dict, axis=1, inplace=True)
                left_joined.set_index(rank, inplace=True)

                avg_df = left_joined[['RA', 'TAX_ID']].groupby(rank).mean()

                # Get observed total.
                observed_total = avg_df["RA"].sum()

                # The difference between 1 and the observed total is assigned higher.
                assigned_higher_ra = 0
                assigned_higher_bool = False

                if pl != "woltka":
                    assigned_higher_ra = 1 - observed_total
                    assigned_higher_bool = assigned_higher_ra > 1e-6

                # Add the assigned higher to the dataframe if it is greater than 1 ppm.
                if assigned_higher_bool:
                    avg_df = add_row(avg_df, rank, "Assigned Higher", assigned_higher_ra, ASSIGNED_HIGHER_ID)

                unclassified_present = 12908 in avg_df["TAX_ID"].values

                unclassified_label = None
                if unclassified_present:
                    unclassified_label = avg_df.loc[avg_df["TAX_ID"] == 12908].index[0]
                
                # Drop the TAX_ID column.
                avg_df = avg_df.drop(columns=["TAX_ID"])

                avg_df, fp_filled = fill_df(avg_df, src, rank, "False Positive")
                # display(pl_df.head(10))
                T = avg_df[["RA"]].T

                display(T)

                T = T.reindex(sorted(T.columns), axis=1)

                reorg_list = [(unclassified_label, unclassified_present, UNCLASS), ("False Positive", fp_filled, FP), ("Assigned Higher", assigned_higher_bool, ASSIGN_HIGHER)]

                T = reorganize_columns(T, reorg_list)
                
                # Change the index to the pipeline name.
                T.index = [name_replace_dict[pl]]

                ax[ax_c].set_xlim(0, 1.01)

                T.plot.barh(alpha=0.5, legend=False, stacked=True, ax=ax[ax_c], fontsize=20)

                fix_bar_colors(ax[ax_c], reorg_list)
                                
                ax[ax_c].set_xlabel("Average Relative Abundance", fontsize=24, labelpad=20)

                ax_c += 1
            
            add_plot_decorations(ax=ax, n_rows=len(groups)+1)
            pdf_output.savefig(fig, bbox_inches="tight", dpi=300)

# stacked_left_join("genus")
stacked_left_join("species")

pdf_output.close()